# CSE 6240 - Web Search & Text Mining - Homework 3

In [1]:
# __author__ = 'Bhanu Verma'
# GTid = '903151012'

In [133]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import linalg
import os, os.path
from scipy.sparse import lil_matrix,linalg
from scipy.sparse import csr_matrix
from scipy.sparse import identity
from scipy.sparse.linalg import norm
from sklearn.cross_validation import train_test_split
from scipy.stats import pearsonr
import pprint
%matplotlib inline

In [3]:
def get_matrix_dimensions():
    doc_size = 0
    vocab_size = 0
    with open(filepath, "r") as f:
        count = 0
        for line in f:
            line = line.rstrip('\n')
            if count==0:
                doc_size = int(line)
            elif count==1:
                vocab_size = int(line)
            else:
                break
                
            count+=1
    
    f.close()
    
    return (doc_size,vocab_size)

In [4]:
def populate_matrix():
    with open(filepath, "r") as f:
        count = 0
        for line in f:
            line = line.rstrip('\n')
            if count > 2:
                arr = line.split()
                i = int(arr[0])-1
                j = int(arr[1])-1
                val = int(arr[2])
                mat[i,j] = val
            count+=1
    f.close()

In [5]:
def get_triplet_list():
    triplet_list = []
    ordered_list = []
    for i in range(n_doc):
        # randomly generate 6 numbers, 1 extra to take care of the case where random number id is equal to i
        rand_arr = np.random.choice(n_doc,6,replace=False)
        doc_list = [] # list of indices of documents
        corr_val = [] # list of calculated correlation coefficients
        tuple_list = []
        
        # fill up correlation values
        for j in range(len(rand_arr)):
            if i!=j:
                a = get_dense_row(i)
                b = get_dense_row(rand_arr[j])
                pear_val = pearsonr(a,b) # it returns a tuple where first element is the coefficient value
                corr_val.append(pear_val[0])
                doc_list.append(rand_arr[j])
                if len(corr_val) == 5:
                    break

        # build up triplets
        for a in range(len(corr_val)):
            for b in range(a+1,len(corr_val)):
                if b < len(corr_val):
                    if corr_val[a] >= corr_val[b]:
                        triplet_list.append((i,doc_list[a],doc_list[b]))
                        tuple_list.append((i,doc_list[a],doc_list[b]))
                    else:
                        triplet_list.append((i,doc_list[b],doc_list[a]))
                        tuple_list.append((i,doc_list[b],doc_list[a]))
        
    return triplet_list

In [6]:
def get_dense_row(index):
    return dense_mat[index].tolist()[0] # fix for the issue, tolist() here returns matrix of rowsize 1

In [7]:
def get_sparse_row(index):
    return mat[index]

In [8]:
def dense(sparse):
    sparse = sparse.todense()
    return sparse[0].tolist()[0]

# Question 2.a

In [9]:
# get matrix dimensions
filepath = 'Data/docword.kos.txt'
size_tup = get_matrix_dimensions()

In [10]:
n_doc = size_tup[0]
n_vocab = size_tup[1]

In [11]:
# populate lil_matrix dimensions
mat = lil_matrix((size_tup[0],size_tup[1]),dtype=np.int32)
populate_matrix()

# convert lil_matrix to csr matrix for faster computation
mat = mat.tocsr()
dense_mat = mat.todense()

In [12]:
triplets = get_triplet_list()

In [126]:
# split triplet array into training and testing data
train,test = train_test_split(triplets, train_size = 0.8, random_state=42)

# Question 1 - OASIS Implementation

In [127]:
w = identity(n_vocab,format='csr')
training_loss_list = []
testing_loss_list = []
c = 0.1

In [128]:
def get_similarity_score(a,w_mat,b):
    result = a*w_mat*b
    return result[0,0]

In [129]:
def get_loss(a_i,a_plus,a_neg,w_mat):
    res_a = get_similarity_score(a_i,w_mat,a_plus.transpose())
    res_b = get_similarity_score(a_i,w_mat,a_neg.transpose())
    val = 1-res_a+res_b
    return max(0,val)

In [131]:
for i in range(len(train)):
    p_i = mat[train[i][0]]
    p_plus = mat[train[i][1]]
    p_neg = mat[train[i][2]]
    
    loss = get_loss(p_i,p_plus,p_neg,w)
    if (i%200 == 0):
        print "number of iterations - ", i
    
    if loss != 0:
        diff = (p_plus - p_neg).transpose()
        ones = np.ones((n_vocab,n_vocab))
        diff_csr = csr_matrix(diff.multiply(ones))
        elements = csr_matrix(p_i.multiply(ones))
        v_i = elements.multiply(diff_csr).transpose()
        norm_val = norm(v_i, 'fro')**2 
        tau = min(c,loss/norm_val)
        w = w + tau*v_i

    train_loss = 0
    test_loss = 0
    
    if (i%200 == 0):
        # calculate Training Loss
        for z in range(len(train)):
            a_i = mat[train[z][0]]
            a_plus = mat[train[z][1]]
            a_neg = mat[train[z][2]]
            train_loss += get_loss(a_i,a_plus,a_neg,w)
        training_loss_list.append(train_loss)
        
        # calculate Testing Loss
        for k in range(len(test)):
            t_i = mat[test[k][0]]
            t_plus = mat[test[k][1]]
            t_neg = mat[test[k][2]]
            test_loss += get_loss(t_i,t_plus,t_neg,w)
        testing_loss_list.append(test_loss)
        
        print "Training Error", train_loss
        print "Testing Error", test_loss
        print "----------------------"

number of iterations -  0
Training Error 42530.0
Testing Error 10909.0
----------------------
number of iterations -  200
Training Error 40098.1773311
Testing Error 10335.4564069
----------------------
number of iterations -  400
Training Error 33387.8171561
Testing Error 8803.28919443
----------------------
number of iterations -  600
Training Error 30364.5545017
Testing Error 8096.29507236
----------------------
number of iterations -  800
Training Error 27877.6359443
Testing Error 7399.93608424
----------------------
number of iterations -  1000
Training Error 26128.0555449
Testing Error 7003.46030494
----------------------
number of iterations -  1200
Training Error 24356.1888519
Testing Error 6570.85029173
----------------------
number of iterations -  1400
Training Error 23454.262254
Testing Error 6356.0038017
----------------------
number of iterations -  1600
Training Error 21388.4052836
Testing Error 5815.10204037
----------------------
number of iterations -  1800
Training Er

In [134]:
print pprint.pprint(training_loss_list)

[42530.0,
 40098.177331061619,
 33387.817156097124,
 30364.554501692241,
 27877.635944294081,
 26128.05554486546,
 24356.188851906772,
 23454.262253972989,
 21388.405283564931,
 19658.998312608554,
 18883.181230535534,
 18248.494286874069,
 17902.254681625229,
 17772.231266294948,
 17432.226318449615,
 16961.609762017557,
 16437.504079654715,
 15627.193226453091,
 15383.153094131363,
 15025.815513011954,
 14467.292833994228,
 14377.128679807051,
 13856.462950193596,
 13673.738715903562,
 13364.540625348363,
 13208.955923640864,
 12918.35938151508,
 12732.889858752591,
 12642.192345355597,
 12249.565188701963,
 12158.520086301609,
 12003.353851238506,
 11747.178464726565,
 11642.058446743717,
 11474.917239790388,
 11308.143007670938,
 11221.596503119852,
 11143.390210226769,
 11215.989555558117,
 11116.787316227592,
 11280.723778625234,
 10910.039868999811,
 10798.511473633627,
 10698.499199093367,
 10493.654212586118,
 10421.265003849188,
 10306.317408840125,
 10320.044798223111,
 1028

In [135]:
print pprint.pprint(testing_loss_list)

[10909.0,
 10335.456406878948,
 8803.2891944251824,
 8096.2950723578579,
 7399.9360842430715,
 7003.4603049397501,
 6570.8502917293308,
 6356.003801697987,
 5815.1020403679067,
 5375.2122550385611,
 5230.7737412205133,
 5098.8972515182641,
 5017.6633605813686,
 4981.7172129640849,
 4898.1416437680518,
 4783.0636953747235,
 4641.2350702722806,
 4485.2700142520998,
 4414.1633436563407,
 4349.565736417454,
 4223.1292275128071,
 4207.9367823078019,
 4102.9218199707866,
 4072.268361900954,
 4029.9835985119093,
 4014.2598287018504,
 3979.6842693721865,
 3936.8547471004281,
 3930.6439585091703,
 3863.5375700256059,
 3831.3702888116463,
 3830.0965850867074,
 3784.3973716506725,
 3798.5531830370769,
 3760.7553449628372,
 3724.573768231156,
 3730.7926343988438,
 3729.3363233593614,
 3754.0001977743191,
 3752.8573755387374,
 3790.0447158581565,
 3742.3001421648582,
 3721.9433839129797,
 3717.9536598837426,
 3687.4536267399221,
 3677.2838468597943,
 3662.4730693873103,
 3657.2658713173264,
 3674.0

In [ ]:
a = mat[train[i][1]]